# Module 1 Homework

In [1]:
# install main library YFinance
try:
    import yfinance as yf
except:
    !pip install yfinance
    import yfinance as yf

In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date


# Question 1: [Macro] Average growth of GDP in 2023

What is the average growth (in %) of GDP in 2023?

In [3]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 4; day=20
Period for indexes: 1954-04-20 to 2024-04-20 


In [4]:
# Real Potential Gross Domestic Product (GDPPOT), Billions of Chained 2012 Dollars, QUARTERLY
# https://fred.stlouisfed.org/series/GDPPOT
gdppot = pdr.DataReader("GDPPOT", "fred", start=start)

In [5]:
gdppot['gdppot_us_yoy'] = gdppot.GDPPOT/gdppot.GDPPOT.shift(4)-1
gdppot

,GDPPOT,gdppot_us_yoy
DATE,,
1954-07-01,2922.682483,NaN
1954-10-01,2941.203176,NaN
1955-01-01,2959.166643,NaN
1955-04-01,2978.435270,NaN
1955-07-01,2997.302681,0.025531
...,...,...
2023-04-01,22246.230000,0.021312
2023-07-01,22369.710000,0.021719
2023-10-01,22495.190000,0.022190


In [6]:
round(gdppot[gdppot.index.year==2023]['gdppot_us_yoy'].mean()*100, 1)

2.2

# Question 2. [Macro] Inverse "Treasury Yield"

Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.

In [7]:
start = '2001-01-01'

dgs2 = pdr.DataReader("DGS2", "fred", start=start)
dgs10 = pdr.DataReader("DGS10", "fred", start=start)

dgs_2_10 = pd.concat([dgs2,  dgs10], axis=1)
round((dgs_2_10['DGS10'] - dgs_2_10['DGS2']).min(), 1)

-1.1

# Question 3. [Index] Which Index is better recently?

Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)

In [8]:
# INDEXES from Yahoo Finance
# DAX index (XETRA - XETRA Delayed Price. Currency in EUR)
# WEB: https://finance.yahoo.com/quote/%5EGDAXI

start = '2019-04-09'
end = '2024-04-09'

GSPC_index = yf.download(tickers = "^GSPC",
                     start = start,
                     end = end)

GSPC_index.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-04-09,2886.580078,2886.879883,2873.330078,2878.199951,2878.199951,3032480000
2019-04-10,2881.370117,2889.709961,2879.129883,2888.209961,2888.209961,3092230000
2019-04-11,2891.919922,2893.419922,2881.989990,2888.320068,2888.320068,2970650000
2019-04-12,2900.860107,2910.540039,2898.370117,2907.409912,2907.409912,3726050000
2019-04-15,2908.320068,2909.600098,2896.479980,2905.580078,2905.580078,3114530000


In [9]:
MXX_index = yf.download(tickers = "^MXX",
                     start = start,
                     end = end)

MXX_index.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-04-09,45346.828125,45346.828125,44854.531250,45151.628906,45151.628906,191575000
2019-04-10,45204.750000,45219.410156,44850.109375,44909.140625,44909.140625,145314900
2019-04-11,44872.531250,44966.500000,44373.488281,44580.058594,44580.058594,109090000
2019-04-12,44767.671875,44888.699219,44534.378906,44686.058594,44686.058594,143662400
2019-04-15,44649.738281,44900.929688,44347.531250,44625.781250,44625.781250,108627100


In [10]:
sp500_start_value = GSPC_index['Close'].iloc[0]
sp500_end_value = GSPC_index['Close'].iloc[-1]

ipc_mexico_start_value = MXX_index['Close'].iloc[0]
ipc_mexico_end_value = MXX_index['Close'].iloc[-1]

# Calculate the growth ratio for each index
sp500_growth_ratio = sp500_end_value / sp500_start_value
ipc_mexico_growth_ratio = ipc_mexico_end_value / ipc_mexico_start_value

# Calculate the percentage growth for each index
sp500_growth_percent = (sp500_growth_ratio - 1) * 100
ipc_mexico_growth_percent = (ipc_mexico_growth_ratio - 1) * 100

round(sp500_growth_percent), round(ipc_mexico_growth_percent)

(81, 28)

# Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks

Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023

In [11]:
def range_ratio(ticker, start, end):
    adj_Close = yf.download(tickers=ticker, start=start, end=end)['Adj Close']
    return round((adj_Close.max()-adj_Close.min())/adj_Close.max(), 2)

In [13]:
start = '2023-01-01'
end = '2023-12-31' 

top_stocks = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

range_ratio_stocks = [range_ratio(ticker, start, end) for ticker in top_stocks]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [30]:
pd.DataFrame([dict(zip(top_stocks, range_ratio_stocks))]).T

,0
2222.SR,0.21
BRK-B,0.21
AAPL,0.37
MSFT,0.42
GOOG,0.39
JPM,0.28


# Question 5. [Stocks] Dividend Yield

Find the largest dividend yield for the same set of stocks

In [24]:
def div_adjclose(ticker, start, end):
    epi = yf.Ticker(ticker)
    dividends = epi.get_actions()['Dividends']
    dividends = dividends[(dividends.index >= start) & (dividends.index <= end)]
    adj_Close = yf.download(tickers=ticker, start=start, end=end)['Adj Close']
    return round(100*dividends.sum()/adj_Close.iloc[-1], 1)

In [25]:

div_adjclose_stocks = [div_adjclose(ticker, start, end) for ticker in top_stocks]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [29]:
pd.DataFrame([dict(zip(top_stocks, div_adjclose_stocks))]).T

,0
2222.SR,2.8
BRK-B,0.0
AAPL,0.5
MSFT,0.7
GOOG,0.0
JPM,2.4


# Question 6. [Exploratory] Investigate new metrics

Free text answer

In [33]:
def calculate_max_drawdown(ticker, years=20):

    end_date = pd.Timestamp.today()
    start_date = end_date - pd.DateOffset(years=years)
    data = yf.download(ticker, start=start_date, end=end_date)

    data['Returns'] = data['Adj Close'].pct_change()

    data['Wealth_Index'] = (1 + data['Returns']).cumprod()

    data['Previous_Peaks'] = data['Wealth_Index'].cummax()

    data['Drawdown'] = (data['Wealth_Index'] - data['Previous_Peaks']) / data['Previous_Peaks']

    max_drawdown = data['Drawdown'].min()
    return max_drawdown


max_drawdown_sp500 = calculate_max_drawdown("^GSPC")
print(f"The maximum drawdown of the S&P 500 in the last 20 years is: {max_drawdown_sp500*100:.2f}%")


[*********************100%%**********************]  1 of 1 completed

The maximum drawdown of the S&P 500 in the last 20 years is: -56.78%


The maximum drawdown is a critical risk metric for assessing the potential downside of an investment portfolio. It measures the largest peak-to-trough decline in the value of an investment before a new peak is achieved.

The maximum drawdown is essential because it provides a clear measure of the most significant drop an investment has faced from peak to trough, which helps investors understand the risk of substantial losses. For instance, the S&P 500 experienced a maximum drawdown of -56.78% over the last 20 years, highlighting the level of potential financial risk during severe market downturns. This example underlines the importance of the metric for risk assessment and financial planning.

# Question 7. [Exploratory] Time-driven strategy description around earnings releases 

Based on the earnings data, here's a potential strategy to select companies of interest:

- Identify Companies Beating Estimates: Look for companies that reported earnings exceeding analysts expectations in the previous week. These companies might experience a stock price increase following the positive news.

- Focus on Upcoming Reports: From the upcoming earnings calendar, select companies in industries with positive outlooks or recent positive news. Analyze their past performance and analyst ratings.

- Compare Price Movement: Compare the stock price movement of companies exceeding estimates in the previous week with the overall market performance.

By combining information from past and upcoming earnings reports, I can develop a watchlist of companies with potentially significant price movements in the coming weeks.